In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6'

In [2]:
import json
from tqdm.auto import tqdm

In [3]:
import sys
sys.path.append('/home/majed_alshaibani/Projects/jrcai_corekit/src')

In [4]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # suppress requests warnings of InsecureRequestWarning

In [5]:
from llm import *


/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [6]:
llm_loader = LLMLoader(
    '/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/',
    llm_initializer=Llama31Initializer(),
    )
messsage_generator = MessageGeneratorFromLocalLLM(llm_loader)

# Exploring Llama outputs

In [7]:
def get_chat_completion(messages):
    llm_response = messsage_generator(messages)
    return llm_response[0]

test the API

In [8]:
print(get_chat_completion(
    messages=[[{
            'role': 'user',
            'content': 'ما هو لون نجوم السماء؟'
        }]]
    )
)

loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/config.json
Model config LlamaConfig {
  "_name_or_path": "/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.43.3",
  "use_cac

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate 

Meta-Llama-3.1-70B-Instruct - batches sizes = [1]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]

النجوم بيضاء، ولكن يمكن أن تظهر بألوان مختلفة مثل الأحمر أو الأزرق أو الأصفر.


# Generating posts

## utilities functions

In [9]:
def save_posts_to_jsonl(articles, file_path):  
    # Append new articles
    with open(file_path, 'w', encoding='utf-8') as f:
        for post in articles:
            json_object = post
            f.write(json.dumps(json_object, ensure_ascii=False) + '\n')

def load_existing_posts(file_path):
    """Load existing posts from a JSONL file, if it exists."""
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            return [json.loads(line) for line in file]
    return []

In [10]:
def load_posts():
    posts = []
    with open('arabic_datasets/social_media_dataset.json', 'r', encoding='utf-8') as f:
        posts = json.load(f)
    return posts

In [11]:
posts = load_posts()
len(posts)

3500

In [12]:
def generate_posts(generation_prompts, generation_type, batch_size=100):
    # Define file path for saving posts
    os.makedirs("generated_arabic_datasets/llama-batched/arabic_social_media_dataset", exist_ok=True)
    file_path = f"generated_arabic_datasets/llama-batched/arabic_social_media_dataset/{generation_type}_posts_generation.jsonl"
    
    # Load already generated posts to resume if script is interrupted
    generated_posts = load_existing_posts(file_path)
    start_idx = len(generated_posts)
    print(f"Resuming from batch {start_idx // batch_size}...")

    # Prepare batches of prompts starting from the last completed batch
    batched_messages = [
        [
            [{'role': 'user', 'content': prompt}] 
            for prompt in generation_prompts[i:i+batch_size]
        ]
        for i in range(0, len(generation_prompts), batch_size)
    ]
    
    # Main process
    for batch_idx, batch in enumerate(tqdm(batched_messages[start_idx // batch_size:], desc="Processing batches"), start=start_idx // batch_size):
        generated_batch = messsage_generator(batch)
        batch_generated_posts = []
        
        for post in generated_batch:
            post = post.strip().strip('<END>').strip().strip('<START>').strip()
            end_token_index = post.find('<END>')
            if end_token_index > 0:
                post = post[:end_token_index]
            batch_generated_posts.append(post.strip())
        
        # batch_generated_posts = generated_batch

        # Get the current slice of the posts
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, len(generation_prompts))
        
        # Format the results for this batch
        batch_results = [
            {
                "original_post": post,
                "generated_post": generated_post,
            }
            for post, generated_post in zip(
                posts[batch_start:batch_end],
                batch_generated_posts
            )
        ]

        # Extend the accumulated results and save after each batch
        generated_posts.extend(batch_results)
        save_posts_to_jsonl(generated_posts, file_path)

    print(f"All batches processed and saved to {file_path}")

## Generate by polishing posts

In [13]:
generate_by_polishing_post_prompt = """
أنت خبير في تحليل وتحسين منشورات وسائل التواصل الاجتماعي. مهمتك هي تحسين وصقل المنشور المقدم مع الحفاظ على أسلوبه وطابعه الأصلي.

المطلوب تحسين المنشور مع مراعاة:
- الحفاظ على اللهجة المستخدمة إن وجدت
- الإبقاء على التشكيل إن وجد
- استخدام مفردات قريبة من المفردات الأصلية
- تصحيح الأخطاء الإملائية والنحوية دون تغيير جوهري في الأسلوب
- تحسين التماسك والوضوح مع الحفاظ على الطابع العفوي للمنشور
- الحفاظ على عدد كلمات مقارب للمنشور الأصلي ما أمكن وحيثما كان مناسباً

مهم جداً: يجب الالتزام بالتالي بدقة:
- ابدأ المنشور المحسّن مباشرة بعد علامة <START>
- انتهِ المنشور المحسّن بعلامة <END>
- لا تضف أي تعليقات أو ملاحظات قبل <START> أو بعد <END>
- لا تكتب أي عبارات تمهيدية أو ختامية
- لا تضف أي نص ليس جزءاً من المنشور المحسّن نفسه
- لا تعلق على التحسينات أو التغييرات التي قمت بها

المنشور الأصلي:
{post}

المنشور المحسّن:
<START>
""".strip()

In [ ]:
print(generate_by_polishing_post_prompt.format(post=posts[0]))

In [15]:
generate_by_polishing_posts_prompts = []
for post in posts:
    prompt = generate_by_polishing_post_prompt.format(post=post)
    generate_by_polishing_posts_prompts.append(prompt)

In [16]:
len(generate_by_polishing_posts_prompts)

3500

In [ ]:
generate_by_polishing_posts_prompts[:3]

In [18]:
generate_posts(generation_prompts=generate_by_polishing_posts_prompts, generation_type='by_polishing', batch_size=2)

Resuming from batch 1750...


Processing batches: 0it [00:00, ?it/s]

All batches processed and saved to generated_arabic_datasets/llama-batched/arabic_social_media_dataset/by_polishing_posts_generation.jsonl


In [ ]:
exit()

: 